In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import math

In [ ]:
df = pd.read_csv('path/to/your/ID3_example.csv')

In [ ]:
def entropy(labels):
    counts = Counter(labels)
    total = len(labels)
    return -sum((count/total) * math.log2(count/total) for count in counts.values())

In [ ]:
def information_gain(df, feature, target):
    total_entropy = entropy(df[target])
    values = df[feature].unique()
    weighted_entropy = sum((len(df[df[feature] == value]) / len(df)) * entropy(df[df[feature] == value][target]) for value in values)
    return total_entropy - weighted_entropy

In [ ]:
class Node:
    def __init__(self, feature=None, value=None, is_leaf=False, prediction=None):
        self.feature = feature
        self.value = value
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.children = []

def id3(df, features, target):
    labels = df[target]
    if len(set(labels)) == 1:
        return Node(is_leaf=True, prediction=labels.iloc[0])
    if not features:
        return Node(is_leaf=True, prediction=labels.mode()[0])

    gains = [(feature, information_gain(df, feature, target)) for feature in features]
    best_feature, best_gain = max(gains, key=lambda x: x[1])

    root = Node(feature=best_feature)
    for value in df[best_feature].unique():
        subset = df[df[best_feature] == value]
        if subset.empty:
            continue
        child = id3(subset, [f for f in features if f != best_feature], target)
        child.value = value
        root.children.append(child)

    return root

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_df, test_df = train_test_split(df, test_size=0.2)

features = list(train_df.columns)
features.remove('target')
tree = id3(train_df, features, 'target')

def predict(node, sample):
    if node.is_leaf:
        return node.prediction
    feature_value = sample[node.feature]
    for child in node.children:
        if child.value == feature_value:
            return predict(child, sample)
    return None

predictions = test_df.apply(lambda x: predict(tree, x), axis=1)
accuracy = accuracy_score(test_df['target'], predictions)
print(f'Akurasi: {accuracy}')